In [ ]:
from datetime import date, timedelta, datetime

from Stock import *

s = date(2020,1,1)
e = date(2021,12,20)
tesla = Stock("tsla")
tesla.load_data()
#tesla.add_data_range(s,e,stockpath='pricedata/tsla.csv')
#tesla.save_data()


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
  
warnings.filterwarnings(action = 'ignore')
  
import gensim
from gensim.models import Word2Vec
import seaborn as sns

In [ ]:
data = []
for j in tesla.df['text'].values:
    temp = [i.lower() for i in word_tokenize(j)]
    data.append(temp)

In [ ]:
model1 = gensim.models.Word2Vec(data, min_count = 1, 
                              size = 5000, window = 7, workers=5, sg=1)

In [ ]:
word1 = "good"
word2 = "bad"
print("Cosine similarity between '{}' and '{}'".format(word1,word2),
    model1.similarity(word1, word2))

In [ ]:
model1.most_similar('germany')[:5]

In [ ]:
tempdf = tesla.df.copy()
tempdf['label'] = tempdf.apply(
    lambda row : int(row.percent_change > 0)
    ,axis=1)
tempdf.loc[tempdf['percent_change'].isna(), 'label'] = np.nan

In [ ]:
tempdf['label'].hist()

In [ ]:
def word_vectorize(wd):
    if wd in model1.wv.vocab:
        return model1[wd.lower()]
    else:
        return np.zeros(5000)*np.nan
def mean_vectorize(tx):
    temp = np.nanmean([word_vectorize(i) for i in word_tokenize(tx)],axis=0).tolist()
    return temp
dataset = tempdf.dropna()
inputs = dataset[['text']].apply(
    lambda row : mean_vectorize(row.text)
    ,axis=1).tolist()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, dataset['label'].astype(int), test_size=0.3, shuffle=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,)
clf.fit(X_train, y_train)
print(clf.score(X_train,y_train))
print(clf.score(X_test,y_test))

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(max_iter=20000, tol=1e-5,shuffle=True)
sgd.fit(X_train, y_train)
print(sgd.score(X_train,y_train))
print(sgd.score(X_test,y_test))

In [ ]:
tesla.load_data()
tesla.add_data(date.today()-timedelta(days=0))
inputs = tesla.get_data(date.today()-timedelta(days=0))[['text']].apply(
    lambda row : mean_vectorize(row.text)
    ,axis=1).tolist()
# 0 is decrease, 1 is increase

In [ ]:
sns.countplot(clf.predict(inputs))

In [ ]:
sns.countplot(sgd.predict(inputs))

In [ ]:
tesla.get_data(date.today()-timedelta(days=0))